In [ ]:
import numpy as np
import pandas as pd
import faiss
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

# Load the filtered dataset
filtered_df = pd.read_csv('../filtefiltered_complaintsred/filtered_complaints.csv')

# Load embeddings and IDs
embeddings = np.load('vector_store/embeddings.npy')
ids = np.load('vector_store/ids.npy')

# Initialize FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype(np.float32))

# Initialize RAG components
tokenizer = RagTokenizer.from_pretrained("facebook/rag-sequence")
retriever = RagRetriever.from_pretrained("facebook/rag-sequence", index=index)
model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence")

# Function to generate response based on a query
def generate_response(query):
    # Encode the query
    inputs = tokenizer(query, return_tensors="pt")
    
    # Retrieve relevant documents
    retrieved_docs = retriever(inputs['input_ids'], inputs['attention_mask'], return_tensors='pt')
    
    # Generate answer
    outputs = model.generate(**retrieved_docs)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    return answer

query = "What issues are common with credit cards?"
response = generate_response(query)
print("Response:", response)